In [1]:
import canais,\
        cities,\
        data,   \
        etapas,  \
        guideshop,\
        loss,      \
        main,       \
        maps,        \
        outbound,     \
        stages,        \
        tools,          \
        uf,              \
        visions,          \
        campaigns           

REPORT_PATH = 'C:/Users/ArthurRodrigues/Codes/Expansion/Report/'

In [2]:
from unidecode import unidecode #to padronize strings
import pandas as pd #tables working

In [3]:
atendidas_new = pd.read_excel('C:/Users/ArthurRodrigues/Codes/Expansion/Marketing/Cidades_Atendidas.xlsx')
atendidas = atendidas_new.copy() #make a copy to save changes

In [4]:
# Rename the columns to make merges/concats easy
atendidas = atendidas_new.rename(columns={'CIDADES ATENDIDAS':'cidades','Estado':'UF', 'CD OU CDA Responsável':'CDA'}).copy()

In [5]:
# padronize cities
atendidas = atendidas.drop(853) #blanck row
atendidas.loc[:,'cidades'] = atendidas.loc[:,'cidades'].str.lower()
atendidas.loc[:,'cidades'] = atendidas.loc[:,'cidades'].str.strip()
atendidas.loc[:,'cidades'] = atendidas.loc[:,'cidades'].apply(lambda x : unidecode(x))

In [6]:
# passing the object dtypes to string
cols = atendidas.select_dtypes(include='O').columns
atendidas = atendidas.astype({col:'string' for col in cols}).copy()
#atendidas.dtypes

In [7]:
# create a KEY that gives and unique ID for each city
atendidas.loc[:,'KEY'] = list(zip(atendidas.loc[:,'cidades'],atendidas.loc[:,'UF']))
atendidas              = atendidas.astype({'KEY':'string'})

In [8]:
atendidas.head(7)

,cidades,UF,FREQUÊNCIA,DIAS DA SEMANA COM ATENDIMENTO,CIDADE POLO (LOJAS),ROTEIRO,CDA,Prazo de Doca,Prazo para entrega (Last Mile),Prazo Total,KM do CD ou CDA,KM DA CIDADE POLO,DIST. FRANQUIA,CIDADE PÓLO,KEY
0,abre campo,MG,1x Quinzena,Segunda,Abre Campo,6N,CDA Ipatinga,2.0,10.0,12.0,146,0,NÃO,SIM,"('abre campo', 'MG')"
1,santo antonio do grama,MG,1x Quinzena,Segunda,Abre Campo,6N,CDA Ipatinga,2.0,10.0,12.0,0,17.4,NÃO,NÃO,"('santo antonio do grama', 'MG')"
2,sericita,MG,1x Quinzena,Segunda,Abre Campo,6N,CDA Ipatinga,2.0,10.0,12.0,0,26.1,NÃO,NÃO,"('sericita', 'MG')"
3,afonso claudio,ES,1x Semana,Segunda,Afonso Cláudio,3K,CDA Cariacica,2.0,5.0,7.0,143,0,NÃO,SIM,"('afonso claudio', 'ES')"
4,aimores,MG,1x Semana,Quarta,Aimorés,3Z,CDA Cariacica,2.0,5.0,7.0,187,0,NÃO,SIM,"('aimores', 'MG')"
5,alem paraiba,MG,5x Semana,"Seg, Ter, Qua, Qui e Sex",Além Paraíba,17,CDA Muriaé,1.0,2.0,3.0,116,0,SIM,SIM,"('alem paraiba', 'MG')"
6,alfenas,MG,1x Semana,Sexta,Alfenas,5O,CDA Varginha,1.0,5.0,6.0,67.8,0,NÃO,SIM,"('alfenas', 'MG')"


In [9]:
# Cidades Expansão
cex = data.load_pkl(name='cex')
cex.head(4)

,KEY,UF,Região,Cidade Polo,cidades,População,Classif Cidades,Status Expansão,Ação mkt,STATUS MARKETING,...,haveGuide,guidesDesligadas,dontHaveNewGuide,haveCDA,cdCity,Is_cdF,cdaUpdated,cdKey,Is_cdFupdated,Is_Target
0,"('serra da saudade', 'MG')",MG,central mineira,bom despacho,serra da saudade,781.0,0 - Baixo potencial,!Baixo Potencial,<NA>,,...,0,0,0,0,futura,0,futura,"(('serra da saudade', 'MG'), 'futura')",0,0
1,"('bora', 'SP')",SP,assis,assis,bora,838.0,0 - Baixo potencial,!Baixo Potencial,<NA>,,...,0,0,0,0,futura,0,futura,"(('bora', 'SP'), 'futura')",0,0
2,"('araguainha', 'MT')",MT,sudeste mato-grossense,rondonopolis,araguainha,940.0,0 - Baixo potencial,!Baixo Potencial,<NA>,<NA>,...,0,0,0,0,nan,0,nan,"(('araguainha', 'MT'), 'nan')",0,0
3,"('anhanguera', 'GO')",GO,sudeste goiano,catalao,anhanguera,1132.0,0 - Baixo potencial,!Baixo Potencial,<NA>,<NA>,...,0,0,0,0,nan,0,nan,"(('anhanguera', 'GO'), 'nan')",0,0


In [11]:
# New Columns
cex.columns[:]

Index(['KEY', 'UF', 'Região', 'Cidade Polo', 'cidades', 'População',
       'Classif Cidades', 'Status Expansão', 'Ação mkt', 'STATUS MARKETING',
       'STATUS OUTBOUND', 'CD/CDA', 'Loja', ' Pot. Consumo/Ano - R$ ',
       ' Share 10%/Mês - RS ', 'Is_PoloIncluded', 'Is_LogIncluded',
       'Is_RegionIncluded', 'haveGuide', 'guidesDesligadas',
       'dontHaveNewGuide', 'haveCDA', 'cdCity', 'Is_cdF', 'cdaUpdated',
       'cdKey', 'Is_cdFupdated', 'Is_Target'],
      dtype='object')

In [12]:
# merge to se differences
cex_atendidas = cex.merge(atendidas, on='KEY')
cex_atendidas.head(4)

,KEY,UF_x,Região,Cidade Polo,cidades_x,População,Classif Cidades,Status Expansão,Ação mkt,STATUS MARKETING,...,CIDADE POLO (LOJAS),ROTEIRO,CDA,Prazo de Doca,Prazo para entrega (Last Mile),Prazo Total,KM do CD ou CDA,KM DA CIDADE POLO,DIST. FRANQUIA,CIDADE PÓLO
0,"('vespasiano', 'MG')",MG,metropolitana bh,belo horizonte,vespasiano,129765.0,3 - Grande,Fechada,<NA>,3. Campanha encerrada,...,Vespasiano,58,CDA Nova Lima,1.0,3.0,4.0,54.8,0,NÃO,SIM
1,"('paiva', 'MG')",MG,zona da mata,juiz de fora,paiva,1529.0,0 - Baixo potencial,!Baixo Potencial,<NA>,,...,Santos Dumont,22,CDM,1.0,10.0,11.0,0,41.2,NÃO,NÃO
2,"('sete lagoas', 'MG')",MG,metropolitana bh,sete lagoas,sete lagoas,241835.0,3 - Grande,Fechada,,3. Campanha encerrada,...,Sete Lagoas,42,CDA Nova Lima,1.0,3.0,4.0,86,0,NÃO,SIM
3,"('tapirai', 'MG')",MG,oeste de minas,piumhi,tapirai,1875.0,0 - Baixo potencial,!Baixo Potencial,<NA>,,...,Bambuí,1U,CDA Divinópolis,2.0,5.0,7.0,0,24.2,NÃO,NÃO


In [14]:
# how many is merged?
cex_atendidas.shape

(838, 42)

In [15]:
cex.shape

(3244, 28)

In [16]:
atendidas.shape

(853, 15)

In [18]:
# We can see that _atendidas_ has (853-838) = 15  cities that isn't at _cex_
not_in_cex = set(atendidas.loc[:,'KEY'])-set(cex.loc[:,'KEY'])
not_in_cex

{"('alegre', 'MG')",
 "('antunes', 'MG')",
 "('aracruz', 'ES')",
 "('dona euzebia', 'MG')",
 "('floriano', 'PR')",
 "('garuva', 'PR')",
 "('macuco de minas', 'MG')",
 "('nova alexandria', 'SP')",
 "('nova lidice', 'BA')",
 "('nova lima 2', 'MG')",
 "('paraty', 'SP')",
 "('rancho alegre', 'BA')",
 "('retiro do muriae', 'RJ')",
 "('sereno', 'MG')",
 "('taquaral', 'MG')"}

In [22]:
# CD pattern
cex_atendidas.loc[:,['CD/CDA','CDA']].value_counts()

CD/CDA                  CDA                     
cda mg varginha         CDA Varginha                51
cda mg nova lima        CDA Nova Lima               45
futura                  CDA Muriaé                  42
cda mg muriae           CDA Muriaé                  42
cda sp guarulhos        CDA Sumaré                  42
                                                    ..
f1.cda pr londrina      CDA São José dos Pinhais     1
cda mg nova lima        CDA Ipatinga                 1
cda rj resende          CDA Duque de Caxias          1
cda rj duque de caxias  CDA Muriaé                   1
nan                     CDA Uberlândia               1
Name: count, Length: 65, dtype: int64

In [26]:
# So we can see that MUST be updated
cdUpdate     = cex_atendidas.loc[:,['CD/CDA','CDA']].value_counts()
cd_to_update = dict((cdUpdate.index))

# What we have to update
dict((cdUpdate.index).sort_values())

{'cda es cariacica': 'CDA Muriaé',
 'cda mg ipatinga': 'CDA Ipatinga',
 'cda mg montes claros': 'CDA Montes Claros',
 'cda mg muriae': 'CDA Muriaé',
 'cda mg nova lima': 'CDA Nova Lima',
 'cda mg uberlandia': 'CDA Uberlândia',
 'cda mg varginha': 'CDM',
 'cda rj duque de caxias': 'CDA Muriaé',
 'cda rj resende': 'CDM',
 'cda sp guarulhos': 'CDA Sumaré',
 'cda sp jacarei': 'CDA Resende',
 'cda sp marilia': 'CDA Sumaré',
 'cda sp ribeirao preto': 'CDA Varginha',
 'cdm': 'CDM',
 'f1.cda pr curitiba': 'CDA São José dos Pinhais',
 'f1.cda pr londrina': 'CDA São José dos Pinhais',
 'f1.cda sc joinville': 'CDA São José dos Pinhais',
 'f1.cda sc palhoca': 'CDA São José dos Pinhais',
 'f2.cda es jaguare': 'CDA Cariacica',
 'f2.cda go goiania': 'CDA Uberlândia',
 'f2.cda pr cascavel': 'CDA Marília',
 'f3.cda ba barra do choca': 'CDA Montes Claros',
 'f3.cda ms campo grande': 'CDA Marília',
 'f3.cda pr pato branco': 'CDA São José dos Pinhais',
 'futura': 'CDM',
 'nan': 'CDA Uberlândia'}

In [28]:
# We have rows duplicated.
len(set(cex_atendidas.loc[:,'KEY']))

818